## 상대 모멘텀
- 여러개의 종목을 이용하여 투자를 전략
    1. 월초부터 월말까지의 수정 종가를 이용하여 수익율 계산
    2. 월별 수익율이 높은 n개를 선택 
    3. 해당하는 종목들을 매수하여 해장하는 종목의 신호가 없어지면 매도하는 형식으로 수익율 생성

In [1]:
import pandas as pd 
import numpy as np 
import warnings
import os 
from glob import glob
from datetime import datetime

In [2]:
# 경고 메시지 출력 필터 
warnings.filterwarnings('ignore')

In [3]:
# 기준 년월 컬럼과 월별 수익율 컬럼을 생성하는 함수
# 매개변수 5개 
    # 데이터프레임 : 필수
    # ticker : 필수
    # 투자의 시작시간 : 2010-01-01 기본값
    # 투자의 종료시간 : 현재 시간 기본값
    # 기준이 되는 컬럼 : 'Adj Close' 기본값
def create_1m_rtn(
    _df, 
    _ticker, 
    _start = '2010-01-01', 
    _end = datetime.now(), 
    _col = 'Adj Close'
):
    # 복사본 생성 
    df = _df.copy()
    # 'Date'가 컬럼에 존재하면 인덱스로 변환
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)
    # 인덱스를 시계열 변환
    df.index = pd.to_datetime(df.index)
    # 투자의 시작시간과 종료시간으로 인덱스 필터링
    # 기준이되는 컬럼으로 컬럼 필터링 
    df = df.loc[_start : _end, [_col]]
    # 기준년월 컬럼을 생성해서 index에서 년도-월 추출하여 대입
    df['STD-YM'] = df.index.strftime('%Y-%m')
    # 월별 수익율 컬럼을 생성 
    df['1m_rtn'] = 0
    # ticker를 컬럼에 대입 
    df['CODE'] = _ticker
    # 기준년월의 유니크값을 생성
    ym_list = df['STD-YM'].unique()
    return df, ym_list



In [5]:
aapl = pd.read_csv("csv/AAPL.csv")

In [6]:
aapl_df, ym_list = create_1m_rtn(aapl, 'AAPL')

In [7]:
aapl_df

,Adj Close,STD-YM,1m_rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL
2010-01-06,26.402260,2010-01,0,AAPL
2010-01-07,26.353460,2010-01,0,AAPL
2010-01-08,26.528664,2010-01,0,AAPL
...,...,...,...,...
2019-06-18,198.449997,2019-06,0,AAPL
2019-06-19,197.869995,2019-06,0,AAPL
2019-06-20,199.460007,2019-06,0,AAPL


In [8]:
ym_list

array(['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06',
       '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12',
       '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
       '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
       '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
       '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
       '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
       '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
       '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
       '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
       '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06',
       '2016-07', '2016-08', '2016-09', '2016-10', 

In [9]:
aapl_df.loc[ym_list[1], ].iloc[-1, 0] / \
    aapl_df.loc[ym_list[1], ].iloc[0, 0]

np.float64(1.0507885741373026)

In [12]:
# 특정 위치의 파일의 목록을 로드 하는 기능 
os.listdir('csv/')

['AAPL.csv', 'AMZN.csv', 'GM.csv', 'MSFT.csv', 'USM.csv']

In [26]:
files = glob("./주식data/*.csv")[:10]

In [27]:
files

['./주식data\\BGF리테일.csv',
 './주식data\\BNK금융지주.csv',
 './주식data\\CJ.csv',
 './주식data\\CJ대한통운.csv',
 './주식data\\CJ제일제당.csv',
 './주식data\\DB손해보험.csv',
 './주식data\\DGB금융지주.csv',
 './주식data\\DL.csv',
 './주식data\\DL이앤씨.csv',
 './주식data\\F&F.csv']

- 반복문을 이용 
1. data 폴더에 있는 파일들을 로드
2. create_1m_rtn() 함수에 대입 ( 데이터프레임, 파일명 )
    - price_df, ym_list 변수에 대입
3. price_df를 새로운 데이터프레임(stock_df)에 단순 행 결합  
4. 기준년월의 유니크 값을 이용하여 반복문 생성
    - 기준년월의 데이터를 이용하여 인덱스 필터를 하여 월초, 월말의 데이터를 이용해서 월별 수익율 계산
    - price_df에 월별 수익율을 대입
    - price_df에 월말의 행을 새로운 데이터프레임(month_last_df) 단순 행결합

In [28]:
# 비어있는 데이터프레임 생성 
stock_df = pd.DataFrame()
month_last_df = pd.DataFrame()

# files를 이용해서 반복문을 생성 
for file in files:
    # file : 특정 경로와 파일명
    # print(file)
    # 경로 파일명을 나눠준다. 
    folder, name = os.path.split(file)
    # 파일명에서 이름과 확장자로 나눠준다. 
    head, tail = os.path.splitext(name)
    # head는 create_1m_rtn에서 _ticker 매개변수에 넣어줄 인자값

    # 데이터프레임 로드 
    read_df = pd.read_csv(file)
    # 함수 호출 
    price_df, ym_list = create_1m_rtn(read_df, head)
    # price_df를 stock_df에 단순 행 결합
    stock_df = pd.concat([stock_df, price_df], axis=0)

    # 두번째 반복문 생성 
    # 월별 수익율을 계산하여 대입
    for ym in ym_list:
        # ym : 기준년월
        # 월초의 가격 (매수)
        buy = price_df.loc[ym, ].iloc[0, 0]
        # 월말의 가격 (매도)
        sell = price_df.loc[ym, ].iloc[-1, 0]
        # 수익율 생성
        rtn = sell / buy
        # 수익율 대입 
        price_df.loc[ym, '1m_rtn'] = rtn
        # 월말의 데이터를 month_last_df에 단순 행 결합 
        last_df = price_df.loc[ym, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat(
            [month_last_df, last_df], axis=0)
            



In [29]:
month_last_df['CODE'].unique()

array(['BGF리테일', 'BNK금융지주', 'CJ', 'CJ대한통운', 'CJ제일제당', 'DB손해보험', 'DGB금융지주',
       'DL', 'DL이앤씨', 'F&F'], dtype=object)

In [32]:
# 백업 
month_rtn_df = month_last_df.copy()

In [33]:
# 인덱스를 리셋 
month_rtn_df.reset_index(inplace=True)

In [34]:
# 테이블을 재구조화 
month_rtn_df = month_rtn_df.pivot_table(
    index = '날짜', 
    columns = 'CODE', 
    values = '1m_rtn'
)

In [35]:
month_rtn_df.head(1)

CODE,BGF리테일,BNK금융지주,CJ,CJ대한통운,CJ제일제당,DB손해보험,DGB금융지주,DL,DL이앤씨,F&F
날짜,,,,,,,,,,
2015-01-30,NaN,0.961777,1.095242,1.021333,1.161716,0.969035,0.946429,0.852719,NaN,NaN


In [36]:
# month_rtn_df의 데이터들을 랭크화(열의 값들은 이용)
month_rtn_df = month_rtn_df.rank(axis=1, 
    ascending=False, 
    pct=True)

In [37]:
# 상위의 40% 종목을 선택 
# where() 함수를 사용 
    # where (조건식, 거짓일때 대입된 데이터)
month_rtn_df = \
    month_rtn_df.where( month_rtn_df <= 0.4, 0 )

In [38]:
# 0이 아닌 데이터들은 1로 변환
month_rtn_df[month_rtn_df != 0] =1

In [39]:
month_rtn_df.head(1)

CODE,BGF리테일,BNK금융지주,CJ,CJ대한통운,CJ제일제당,DB손해보험,DGB금융지주,DL,DL이앤씨,F&F
날짜,,,,,,,,,,
2015-01-30,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# stock_df의 code의 unique()를 변수에 저장 
stock_codes = stock_df['CODE'].unique()

In [42]:
flag_col = month_rtn_df.loc['2015-01-30',] == 1
month_rtn_df.loc['2015-01-30' , flag_col].index

Index(['CJ', 'CJ제일제당'], dtype='object', name='CODE')

In [43]:
# 해당 일자의 구매하려는 종목들을 딕셔너리 생성 
sig_dict = dict()

for idx in month_rtn_df.index:
    # idx : month_rtn_df의 인덱스들 (시계열)
    flag_col = month_rtn_df.loc[idx, ] == 1
    ticker_list = list(
        month_rtn_df.loc[idx, flag_col].index
    )
    # sig_dict에 추가
    sig_dict[idx] = ticker_list

In [44]:
sig_dict

{Timestamp('2015-01-30 00:00:00'): ['CJ', 'CJ제일제당'],
 Timestamp('2015-02-27 00:00:00'): ['DGB금융지주', 'DL'],
 Timestamp('2015-03-31 00:00:00'): ['CJ대한통운', 'CJ제일제당'],
 Timestamp('2015-04-30 00:00:00'): ['CJ제일제당', 'DL'],
 Timestamp('2015-05-29 00:00:00'): ['CJ', 'DGB금융지주'],
 Timestamp('2015-06-30 00:00:00'): ['CJ', 'DB손해보험'],
 Timestamp('2015-07-31 00:00:00'): ['CJ', 'CJ제일제당'],
 Timestamp('2015-08-31 00:00:00'): ['BNK금융지주', 'CJ대한통운'],
 Timestamp('2015-09-30 00:00:00'): ['CJ대한통운', 'DB손해보험'],
 Timestamp('2015-10-30 00:00:00'): ['DB손해보험', 'DL'],
 Timestamp('2015-11-30 00:00:00'): ['CJ', 'CJ대한통운'],
 Timestamp('2015-12-30 00:00:00'): ['CJ제일제당', 'DB손해보험'],
 Timestamp('2016-01-29 00:00:00'): ['CJ', 'DL'],
 Timestamp('2016-02-29 00:00:00'): ['DB손해보험', 'DL'],
 Timestamp('2016-03-31 00:00:00'): ['BNK금융지주', 'DB손해보험'],
 Timestamp('2016-04-29 00:00:00'): ['CJ', 'CJ제일제당'],
 Timestamp('2016-05-31 00:00:00'): ['CJ대한통운', 'DB손해보험'],
 Timestamp('2016-06-30 00:00:00'): ['CJ대한통운', 'CJ제일제당'],
 Timestamp('2016-0

In [46]:
# stock_df를 재구조화 
stock_c_matrix = stock_df.reset_index().pivot_table(
    index = '날짜', 
    columns = 'CODE', 
    values = stock_df.columns[0]
)

In [47]:
stock_c_matrix.head()

CODE,BGF리테일,BNK금융지주,CJ,CJ대한통운,CJ제일제당,DB손해보험,DGB금융지주,DL,DL이앤씨,F&F
날짜,,,,,,,,,,
2015-01-02,NaN,13840.0,148517.0,187500.0,303000.0,54900.0,11200.0,58283.0,NaN,NaN
2015-01-05,NaN,13647.0,150403.0,182500.0,315500.0,54100.0,10900.0,56024.0,NaN,NaN
2015-01-06,NaN,13503.0,149460.0,181500.0,321000.0,54000.0,10600.0,52590.0,NaN,NaN
2015-01-07,NaN,12975.0,149460.0,183500.0,320000.0,53300.0,10650.0,52590.0,NaN,NaN
2015-01-08,NaN,13359.0,154646.0,184500.0,323000.0,53400.0,11250.0,53042.0,NaN,NaN


In [48]:
# 거래내역 컬럼을 추가하는 함수 생성 
def create_trade_book(_df):
    # 복사본 생성 
    df = _df.copy()
    # 컬럼의 목록을 list로 생성 
    codes = list(df.columns)
    for code in codes:
        df[f"p_{code}"] = ""
        df[f"r_{code}"] = ""
    return df

In [49]:
book = create_trade_book(stock_c_matrix)

In [50]:
book.head(2)

CODE,BGF리테일,BNK금융지주,CJ,CJ대한통운,CJ제일제당,DB손해보험,DGB금융지주,DL,DL이앤씨,F&F,...,p_DB손해보험,r_DB손해보험,p_DGB금융지주,r_DGB금융지주,p_DL,r_DL,p_DL이앤씨,r_DL이앤씨,p_F&F,r_F&F
날짜,,,,,,,,,,,,,,,,,,,,,
2015-01-02,NaN,13840.0,148517.0,187500.0,303000.0,54900.0,11200.0,58283.0,NaN,NaN,...,,,,,,,,,,
2015-01-05,NaN,13647.0,150403.0,182500.0,315500.0,54100.0,10900.0,56024.0,NaN,NaN,...,,,,,,,,,,


In [33]:
# sig_dict을 이용해서 구매 대기 상태 입력 


In [51]:
for i in sig_dict:
    print(i)
    print(sig_dict[i])
    break

2015-01-30 00:00:00
['CJ', 'CJ제일제당']


In [52]:
sig_dict.items()

dict_items([(Timestamp('2015-01-30 00:00:00'), ['CJ', 'CJ제일제당']), (Timestamp('2015-02-27 00:00:00'), ['DGB금융지주', 'DL']), (Timestamp('2015-03-31 00:00:00'), ['CJ대한통운', 'CJ제일제당']), (Timestamp('2015-04-30 00:00:00'), ['CJ제일제당', 'DL']), (Timestamp('2015-05-29 00:00:00'), ['CJ', 'DGB금융지주']), (Timestamp('2015-06-30 00:00:00'), ['CJ', 'DB손해보험']), (Timestamp('2015-07-31 00:00:00'), ['CJ', 'CJ제일제당']), (Timestamp('2015-08-31 00:00:00'), ['BNK금융지주', 'CJ대한통운']), (Timestamp('2015-09-30 00:00:00'), ['CJ대한통운', 'DB손해보험']), (Timestamp('2015-10-30 00:00:00'), ['DB손해보험', 'DL']), (Timestamp('2015-11-30 00:00:00'), ['CJ', 'CJ대한통운']), (Timestamp('2015-12-30 00:00:00'), ['CJ제일제당', 'DB손해보험']), (Timestamp('2016-01-29 00:00:00'), ['CJ', 'DL']), (Timestamp('2016-02-29 00:00:00'), ['DB손해보험', 'DL']), (Timestamp('2016-03-31 00:00:00'), ['BNK금융지주', 'DB손해보험']), (Timestamp('2016-04-29 00:00:00'), ['CJ', 'CJ제일제당']), (Timestamp('2016-05-31 00:00:00'), ['CJ대한통운', 'DB손해보험']), (Timestamp('2016-06-30 00:00:00'), ['CJ대한통운', 

In [53]:
for i, j in sig_dict.items():
    # i = sig_dict.items()[0]
    print(i)
    print(j)
    break

2015-01-30 00:00:00
['CJ', 'CJ제일제당']


In [54]:
# sig_dict를 이용해서 구매 전 준비내역을 추가 
for date, codes in sig_dict.items():
    # date : key -> 시계열 데이터 (말일 데이터)
    # codes : value -> 종목 리스트 
    # codes 반복문 생성 
    for code in codes:
        # print(code)
        # book에서 인덱스가 date인 컬럼이 r_code인 컬럼에 준비 내역을 추가
        book.loc[date, f"p_{code}"] = f"ready_{code}" 
    # break

In [56]:
book['p_CJ'].value_counts()

p_CJ
            2414
ready_CJ      44
Name: count, dtype: int64

In [57]:
# 보유 내역을 추가하는 함수 생성 
def create_trading(_df, _codes):
    buy_phase = False
    df = _df.copy()
    std_ym = ""

    # _codes : 종목 리스트 -> 컬럼의 이름
    # 종목별로 순회(컬럼)하는 반복문 생성 
    for code in _codes: # 10번 반복
        # 인덱스를 기준으로 반복문 생성
        for idx in df.index:
            # 특정 종목의 포지션을 잡는다. 
            # (전행의 p_code 컬럼이 ready)이고
            # (현재 행의 p_code 컬럼이 "")
            if (df.loc[idx, f"p_{code}"] == "") & \
                (df.shift().loc[idx, f"p_{code}"] == f"ready_{code}"):
                std_ym = idx.strftime('%Y-%m')
                buy_phase = True
            # 구매 조건 : (현재 p_code가 "")이고 
            #            (index의 년도-월과 std_ym 같고)
            #            (buy_phase)
            if (df.loc[idx, f"p_{code}"] == "") & \
                (std_ym == idx.strftime('%Y-%m')) & \
                    (buy_phase):
                    df.loc[idx, f"p_{code}"] = f"buy_{code}"
            # buy_phase, std_ym 초기화
            if df.loc[idx, f"p_{code}"] == "":
                buy_phase = False
                std_ym = ""
    return df


In [58]:
book2 = create_trading(book, stock_codes)

In [59]:
book2['p_CJ'].value_counts()

p_CJ
            1555
buy_CJ       859
ready_CJ      44
Name: count, dtype: int64

In [60]:
for code in stock_codes:
    book2[f"r_{code}"] = ""

In [61]:
book2.head(1)

CODE,BGF리테일,BNK금융지주,CJ,CJ대한통운,CJ제일제당,DB손해보험,DGB금융지주,DL,DL이앤씨,F&F,...,p_DB손해보험,r_DB손해보험,p_DGB금융지주,r_DGB금융지주,p_DL,r_DL,p_DL이앤씨,r_DL이앤씨,p_F&F,r_F&F
날짜,,,,,,,,,,,,,,,,,,,,,
2015-01-02,NaN,13840.0,148517.0,187500.0,303000.0,54900.0,11200.0,58283.0,NaN,NaN,...,,,,,,,,,,


In [62]:
# 수익율 계산하는 함수 생성 
def multi_return(_df, _codes):
    # 복사본 생성 
    df = _df.copy()
    rtn = 1
    # 매수가 dict 형태로 구성 
    buy_dict = dict()
    # 매도가 dict 형태로 구성
    sell_dict = dict()

    # index를 기준으로 반복문 생성 -> 날짜별 매수, 매도 확인 
    for idx in df.index:
        # 종목별로 매수, 매도를 확인 
        for code in _codes:
            # 매수의 조건 : 2행 전(shift(2))의 p_code가 ""
            #             1행 전(shift())의 p_code가 'ready_code"
            #             현재 행의 p_code가 'buy_code'
            if (df.shift(2).loc[idx, f"p_{code}"] == "") & \
                (df.shift().loc[idx, f"p_{code}"] == f"ready_{code}") & \
                    (df.loc[idx, f"p_{code}"] == f"buy_{code}"):
                    # 매수가  -> idx 행에 code 컬럼에 존재
                    buy_dict[code] = df.loc[idx, code]
                    print(f"매수일 : {idx}, 매수 종목 : {code}, 매수가 : {df.loc[idx, code]}")
            # 매도의 조건 : 1행 전의 p_code가 buy_code
            #              현재 행의 p_code가 ""
            elif (df.shift().loc[idx, f"p_{code}"] == f"buy_{code}") & \
                (df.loc[idx, f"p_{code}"] == ""):
                # 매도가 -> idx 행에 code 컬럼에 존재 
                sell_dict[code] = df.loc[idx, code]
                # 수익율 계산
                rtn = sell_dict[code] / buy_dict[code]
                df.loc[idx, f"r_{code}"] = rtn
                print(f"매도일 : {idx}, 매도 종목 : {code}, 매도가 : {sell_dict[code]}, 수익율 : {rtn}")
            # buy_dict, sell_dict의 code 안에 매수가 매도가 초기화
            if df.loc[idx, f"p_{code}"] == "":
                buy_dict[code] = 0
                sell_dict[code] = 0
    return df



In [63]:
rtn_book = multi_return(book2, stock_codes)

매수일 : 2015-02-02 00:00:00, 매수 종목 : CJ, 매수가 : 159361.0
매수일 : 2015-02-02 00:00:00, 매수 종목 : CJ제일제당, 매수가 : 354000.0
매도일 : 2015-03-02 00:00:00, 매도 종목 : CJ, 매도가 : 154175.0, 수익율 : 0.9674575335245136
매도일 : 2015-03-02 00:00:00, 매도 종목 : CJ제일제당, 매도가 : 351000.0, 수익율 : 0.9915254237288136
매수일 : 2015-03-02 00:00:00, 매수 종목 : DGB금융지주, 매수가 : 11650.0
매수일 : 2015-03-02 00:00:00, 매수 종목 : DL, 매수가 : 61988.0
매수일 : 2015-04-01 00:00:00, 매수 종목 : CJ대한통운, 매수가 : 195000.0
매수일 : 2015-04-01 00:00:00, 매수 종목 : CJ제일제당, 매수가 : 372000.0
매도일 : 2015-04-01 00:00:00, 매도 종목 : DGB금융지주, 매도가 : 12050.0, 수익율 : 1.0343347639484979
매도일 : 2015-04-01 00:00:00, 매도 종목 : DL, 매도가 : 57560.0, 수익율 : 0.928566819384397
매도일 : 2015-05-04 00:00:00, 매도 종목 : CJ대한통운, 매도가 : 199500.0, 수익율 : 1.023076923076923
매수일 : 2015-05-04 00:00:00, 매수 종목 : DL, 매수가 : 75904.0
매수일 : 2015-06-01 00:00:00, 매수 종목 : CJ, 매수가 : 234327.0
매도일 : 2015-06-01 00:00:00, 매도 종목 : CJ제일제당, 매도가 : 420500.0, 수익율 : 1.1303763440860215
매수일 : 2015-06-01 00:00:00, 매수 종목 : DGB금융지주, 매수가 : 13000.0
매도일

In [67]:
rtn_book['r_F&F'].value_counts()

r_F&F
            2447
1.480203       1
0.936508       1
0.910152       1
0.870307       1
0.84375        1
1.042105       1
0.950331       1
0.838593       1
0.79496        1
1.002865       1
1.049488       1
Name: count, dtype: int64

In [64]:
# 누적 수익율 계산 
def multi_acc_rtn(_df, _codes):
    # 복사본 생성 
    df = _df.copy()
    acc_rtn = 1

    # 인덱스를 기준으로 반복문 생성
    for idx in df.index:
        count = 0
        rtn = 0
        for code in _codes:
            # 수익율이 존재하는가?
            if df.loc[idx, f"r_{code}"]:
                # 존재하는 경우
                count += 1
                rtn += df.loc[idx, f"r_{code}"]
        if (rtn != 0) & (count != 0):
            acc_rtn *= rtn / count
            print(f"""누적 - 
            매도일 : {idx}, 매도 종목수 : {count}, 
            수익율 : {round( rtn / count , 2 )}""")
        df.loc[idx, 'acc_rtn'] = acc_rtn
    return df, acc_rtn


In [48]:
# if 조건식 
    # 조건식의 타입은? bool
# if 뒤에 조건식이 타입이 bool이 아니라면? -> 강제로 형태를 bool 변환
bool( [1] )

True

In [65]:
multi_df, acc_rtn = multi_acc_rtn(rtn_book, stock_codes)

누적 - 
            매도일 : 2015-03-02 00:00:00, 매도 종목수 : 2, 
            수익율 : 0.98
누적 - 
            매도일 : 2015-04-01 00:00:00, 매도 종목수 : 2, 
            수익율 : 0.98
누적 - 
            매도일 : 2015-05-04 00:00:00, 매도 종목수 : 1, 
            수익율 : 1.02
누적 - 
            매도일 : 2015-06-01 00:00:00, 매도 종목수 : 2, 
            수익율 : 1.04
누적 - 
            매도일 : 2015-07-01 00:00:00, 매도 종목수 : 1, 
            수익율 : 0.91
누적 - 
            매도일 : 2015-08-03 00:00:00, 매도 종목수 : 1, 
            수익율 : 1.0
누적 - 
            매도일 : 2015-09-01 00:00:00, 매도 종목수 : 2, 
            수익율 : 1.02
누적 - 
            매도일 : 2015-10-01 00:00:00, 매도 종목수 : 1, 
            수익율 : 1.03
누적 - 
            매도일 : 2015-11-02 00:00:00, 매도 종목수 : 1, 
            수익율 : 1.04
누적 - 
            매도일 : 2015-12-01 00:00:00, 매도 종목수 : 2, 
            수익율 : 1.0
누적 - 
            매도일 : 2016-01-04 00:00:00, 매도 종목수 : 2, 
            수익율 : 0.94
누적 - 
            매도일 : 2016-02-01 00:00:00, 매도 종목수 : 2, 
            수익율 : 1.05
누적 - 
            매도일 : 2016-0

In [66]:
acc_rtn

np.float64(0.13731407130546414)